Javascript (JS) is the programming language of the web. Because of this role there are a number of high quality ploting libraries with an interactive focus written in JS. One of the most powerful of these libraries is [D3](https://d3js.org) (Data-driven Documents). The learning curve can be quite rough so we will instead use [D3plus](http://d3plus.org) which simplies some of the work. In order to plot JS in jupyter we can use IFrames. First we set up a function to do this.

In [20]:
from IPython.display import IFrame 

count=0
def serve_html(s,w,h):
    import os
    global count
    count+=1
    fn= '__tmp'+str(os.getpid())+'_'+str(count)+'.html'
    with open(fn,'w') as f:
        f.write(s)
    return IFrame('files/'+fn,w,h)

def f1(w=800,h=400):
    d={
       'width'      :w,
       'height'     :h
       }
    with open('f1.template') as f:
        s=f.read()        
    return serve_html(s,w+30,h+30)

The above function f1, takes an object called "f1.template" and serves it. Now we need to plot something in JS and save it as f1.template. Here we use an example from the D3plus [examples](http://d3plus.org/examples/basic/9034389/)

In [21]:
%%writefile f1.template
<!doctype html>
<meta charset="utf-8">

<!-- load D3js -->
<script src="js/d3.js"></script>
//<script src="//d3plus.org/js/d3.js"></script>

<!-- load D3plus after D3js -->
<script src="js/d3plus.js"></script>
//<script src="//d3plus.org/js/d3plus.js"></script>

<!-- create container element for visualization -->
<div id="viz"></div>

<script>
  // create list of node connections
  var connections = [
    {"source": "alpha", "target": "beta"},
    {"source": "alpha", "target": "gamma"},
    {"source": "beta", "target": "delta"},
    {"source": "beta", "target": "epsilon"},
    {"source": "zeta", "target": "gamma"},
    {"source": "theta", "target": "gamma"},
    {"source": "eta", "target": "gamma"}
  ]
  // instantiate d3plus
  var visualization = d3plus.viz()
    .container("#viz")  // container DIV to hold the visualization
    .type("rings")      // visualization type
    .edges(connections) // list of node connections
    .focus("alpha")     // ID of the initial center node
    .draw()             // finally, draw the visualization!
</script>

Overwriting f1.template


Now let's plot the figure specifying the height and width.

In [22]:
f1(w=800,h=600)

Now we can try it on some real data. This example uses a dataset showing which biometricians work with which teams with a rough measure of the strength of this relationship. As the data is stored in a csv we need to convert it to the json format that we'll use in code. Again we write a function to make this easier if we need to repeat this process.

In [23]:
import sys, getopt
import csv
import json
def read_csv(file, json_file, format):
    csv_rows = []
    with open(file) as csvfile:
        reader = csv.DictReader(csvfile)
        title = reader.fieldnames
        for row in reader:
            csv_rows.extend([{title[i]:row[title[i]] for i in range(len(title))}])
        write_json(csv_rows, json_file, format)

#Convert csv data into json and write it
def write_json(data, json_file, format):
    with open(json_file, "w") as f:
        if format == "pretty":
            f.write(json.dumps(data, sort_keys=False, indent=4, separators=(',', ': '),encoding="utf-8",ensure_ascii=False))
        else:
            f.write(json.dumps(data))

Read in the csv from the data subfolder and save the json file to the same directory. Note that we nest the visualisation within the d3.json function.

In [24]:
read_csv("data/Biometrics_work.csv","data/Biometrics_work.json",format=' ')

In [25]:
%%writefile f1.template
<!doctype html>
<meta charset="utf-8">


<script src="js/jquery.min.js"></script>
<!-- load D3js -->
<script src="js/d3.js"></script>
//<script src="//d3plus.org/js/d3.js"></script>

<!-- load D3plus after D3js -->
<script src="js/d3plus.js"></script>
//<script src="//d3plus.org/js/d3plus.js"></script>

<!-- create container element for visualization -->
<div id="viz"></div>

<script>
//read in json
d3.json("data/Biometrics_work.json", function(connections){

    
  // instantiate d3plus
  var visualization = d3plus.viz()
    .container("#viz")  // container DIV to hold the visualization
    .type("rings")      // visualization type
    .edges({"value":connections,"size":"strength"}) // list of node connections
    .focus("Biometrics")     // ID of the initial center node
    .draw()             // finally, draw the visualization!
  })
</script>

Overwriting f1.template


In [26]:
f1(w=1200,h=1000)

we could also plot the whole network.

In [27]:
%%writefile f1.template
<!doctype html>
<meta charset="utf-8">


//<script src="js/jquery.min.js"></script>
<!-- load D3js -->
<script src="js/d3.js"></script>
//<script src="//d3plus.org/js/d3.js"></script>

<!-- load D3plus after D3js -->
<script src="js/d3plus.js"></script>
//<script src="//d3plus.org/js/d3plus.js"></script>

<!-- create container element for visualization -->
<div id="viz"></div>

<script>
//read in json
d3.json("data/Biometrics_work.json", function(connections){

    
  // instantiate d3plus
  var visualization = d3plus.viz()
    .container("#viz")  // container DIV to hold the visualization
    .type("network")      // visualization type
    .edges(connections) // list of node connections
    .draw()             // finally, draw the visualization!
  })
</script>

Overwriting f1.template


In [28]:
f1(w=1000,h=800)

Another intersting library built on top of D3 is [Vega](https://vega.github.io/vega/). An advanced example from the vega [website](https://vega.github.io/vega/examples/global-development/) is given below. While it takes some work in terms of coding the result shows the countries movement over time without cluttering the figure.

In [29]:
%%writefile f1.template
<!DOCTYPE HTML>
<html>
  <head>
    <title>Vega PreFacet Test</title>
    <script src="js/d3.v4.min.js"></script>
    <script src="js/vega-core.min.js"></script>
    <style>
      body { margin: 10px; font: 14px Helvetica Neue; }
      canvas, svg { border: 1px solid #ccc; }
    </style>
  </head>
  <body>
    <div id="view"></div>
    <script>
var runtimeSpec = vega.parse({
  "$schema": "js/vega-schema.json",
  "width": 800,
  "height": 600,
  "padding": 5,

  "data": [
    {
      "name": "gapminder",
      "url": "data/gapminder.json"
    },
    {
      "name": "clusters",
      "values": [
        {"id": 0, "name": "South Asia"},
        {"id": 1, "name": "Europe & Central Asia"},
        {"id": 2, "name": "Sub-Saharan Africa"},
        {"id": 3, "name": "America"},
        {"id": 4, "name": "East Asia & Pacific"},
        {"id": 5, "name": "Middle East & North Africa"}
      ]
    },
    {
      "name": "country_timeline",
      "source": "gapminder",
      "transform": [
        {"type": "filter", "expr": "timeline && datum.country == timeline.country"},
        {"type": "collect", "sort": {"field": "year"}}
      ]
    },
    {
      "name": "thisYear",
      "source": "gapminder",
      "transform": [
        {"type": "filter", "expr": "datum.year == currentYear"}
      ]
    },
    {
      "name": "prevYear",
      "source": "gapminder",
      "transform": [
        {"type": "filter", "expr": "datum.year == currentYear - stepYear"}
      ]
    },
    {
      "name": "nextYear",
      "source": "gapminder",
      "transform": [
        {"type": "filter", "expr": "datum.year == currentYear + stepYear"}
      ]
    },
    {
      "name": "countries",
      "source": "gapminder",
      "transform": [
        {"type": "aggregate", "groupby": ["country"]}
      ]
    },
    {
      "name": "interpolate",
      "source": "countries",
      "transform": [
        {
          "type": "lookup",
          "from": "thisYear", "key": "country",
          "fields": ["country"], "as": ["this"],
          "default": {}
        },
        {
          "type": "lookup",
          "from": "prevYear", "key": "country",
          "fields": ["country"], "as": ["prev"],
          "default": {}
        },
        {
          "type": "lookup",
          "from": "nextYear", "key": "country",
          "fields": ["country"], "as": ["next"],
          "default": {}
        },
        {
          "type": "formula",
          "as": "target_fertility",
          "expr": "interYear > currentYear ? datum.next.fertility : (datum.prev.fertility||datum.this.fertility)"
        },
        {
          "type": "formula",
          "as": "target_life_expect",
          "expr": "interYear > currentYear ? datum.next.life_expect : (datum.prev.life_expect||datum.this.life_expect)"
        },
        {
          "type": "formula",
          "as": "inter_fertility",
          "expr": "interYear==2000 ? datum.this.fertility : datum.this.fertility + (datum.target_fertility-datum.this.fertility) * abs(interYear-datum.this.year)/5"
        },
        {
          "type": "formula",
          "as": "inter_life_expect",
          "expr": "interYear==2000 ? datum.this.life_expect : datum.this.life_expect + (datum.target_life_expect-datum.this.life_expect) * abs(interYear-datum.this.year)/5"
        }
      ]
    },
    {
      "name": "trackCountries",
      "on": [
        {"trigger": "active", "toggle": "{country: active.country}"}
      ]
    }
  ],

  "signals": [
    { "name": "minYear", "value": 1955 },
    { "name": "maxYear", "value": 2005 },
    { "name": "stepYear", "value": 5 },
    {
      "name": "active",
      "value": {},
      "on": [
        {"events": "@point:mousedown, @point:touchstart", "update": "datum"},
        {"events": "window:mouseup, window:touchend", "update": "{}"}
      ]
    },
    { "name": "isActive", "update": "active.country" },
    {
      "name": "timeline",
      "value": {},
      "on": [
        {"events": "@point:mouseover", "update": "isActive ? active : datum"},
        {"events": "@point:mouseout", "update": "active"},
        {"events": {"signal": "active"}, "update": "active"}
      ]
    },
    {
      "name": "tX",
      "on": [{
        "events": "mousemove!, touchmove!",
        "update": "isActive ? scale('x', active.this.fertility) : tX"
      }]
    },
    {
      "name": "tY",
      "on": [{
        "events": "mousemove, touchmove",
        "update": "isActive ? scale('y', active.this.life_expect) : tY"
      }]
    },
    {
      "name": "pX",
      "on": [{
        "events": "mousemove, touchmove",
        "update": "isActive ? scale('x', active.prev.fertility) : pX"
      }]
    },
    {
      "name": "pY",
      "on": [{
        "events": "mousemove, touchmove",
        "update": "isActive ? scale('y', active.prev.life_expect) : pY"
      }]
    },
    {
      "name": "nX",
      "on": [{
        "events": "mousemove, touchmove",
        "update": "isActive ? scale('x', active.next.fertility) : nX"
      }]
    },
    {
      "name": "nY",
      "on": [{
        "events": "mousemove, touchmove",
        "update": "isActive ? scale('y', active.next.life_expect) : nY"
      }]
    },
    {
      "name": "thisDist",
      "value": 0,
      "on":[{
        "events": "mousemove, touchmove",
        "update": "isActive ? sqrt(pow(x()-tX, 2) + pow(y()-tY, 2)) : thisDist"
      }]
    },
    {
      "name": "prevDist",
      "value": 0,
      "on":[{
        "events": "mousemove, touchmove",
        "update": "isActive ? sqrt(pow(x()-pX, 2) + pow(y()-pY, 2)): prevDist"
      }]
    },
    {
      "name": "nextDist",
      "value": 0,
      "on":[{
        "events": "mousemove, touchmove",
        "update": "isActive ? sqrt(pow(x()-nX, 2) + pow(y()-nY, 2)) : nextDist"
      }]
    },
    {
      "name": "prevScore",
      "value": 0,
      "on": [{
        "events": "mousemove, touchmove",
        "update": "isActive ? ((pX-tX) * (x()-tX) + (pY-tY) * (y()-tY))/prevDist || -999999 : prevScore"
      }]
    },
    {
      "name": "nextScore",
      "value": 0,
      "on": [{
        "events": "mousemove, touchmove",
        "update": "isActive ? ((nX-tX) * (x()-tX) + (nY-tY) * (y()-tY))/nextDist || -999999 : nextScore"
      }]
    },
    {
      "name": "interYear",
      "value": 1980,
      "on": [{
        "events": "mousemove, touchmove",
        "update": "isActive ? (min(maxYear, currentYear+5, max(minYear, currentYear-5, prevScore > nextScore ? (currentYear - 2.5*prevScore/sqrt(pow(pX-tX, 2) + pow(pY-tY, 2))) : (currentYear + 2.5*nextScore/sqrt(pow(nX-tX, 2) + pow(nY-tY, 2)))))) : interYear"
      }]
    },
    {
      "name": "currentYear",
      "value": 1980,
      "on":[{
        "events": "mousemove, touchmove",
        "update": "isActive ? (min(maxYear, max(minYear, prevScore > nextScore ? (thisDist < prevDist ? currentYear : currentYear-5) : (thisDist < nextDist ? currentYear : currentYear+5)))) : currentYear"
      }]
    }
  ],

  "scales": [
    {
      "name": "x",
      "type": "linear", "nice": true,
      "domain": {"data": "gapminder", "field": "fertility"},
      "range": "width"
    },
    {
      "name": "y",
      "type": "linear", "nice": true, "zero": false,
      "domain": {"data": "gapminder", "field": "life_expect"},
      "range": "height"
    },
    {
      "name": "color",
      "type": "ordinal",
      "domain": {"data": "gapminder", "field": "cluster"},
      "range": "category"
    },
    {
      "name": "label",
      "type": "ordinal",
      "domain": {"data": "clusters", "field": "id"},
      "range": {"data": "clusters", "field": "name"}
    }
  ],

  "axes": [
    {
      "title": "Fertility",
      "orient": "bottom", "scale": "x",
      "grid": true, "tickCount": 5
    },
    {
      "title": "Life Expectancy",
      "orient": "left", "scale": "y",
      "grid": true, "tickCount": 5
    }
  ],

  "legends": [
    {
      "fill": "color",
      "title": "Region",
      "orient": "right",
      "encode": {
        "symbols": {
          "enter": {
            "fillOpacity": {"value": 0.5}
          }
        },
        "labels": {
          "enter": {
            "text": {"scale": "label", "field": "value"}
          }
        }
      }
    }
  ],

  "marks": [
    {
      "type": "text",
      "encode": {
        "update": {
          "text": {"signal": "currentYear"},
          "x": {"value": 300},
          "y": {"value": 300},
          "fill": {"value": "grey"},
          "fillOpacity": {"value": 0.25},
          "fontSize": {"value": 100}
        }
      }
    },
    {
      "type": "text",
      "from": {"data": "country_timeline"},
      "interactive": false,
      "encode": {
        "enter": {
          "x": {"scale": "x", "field": "fertility", "offset": 5},
          "y": {"scale": "y", "field": "life_expect"},
          "fill": {"value": "#555"},
          "fillOpacity": {"value": 0.6},
          "text": {"field": "year"}
        }
      }
    },
    {
      "type": "line",
      "from": {"data": "country_timeline"},
      "encode": {
        "update": {
          "x": {"scale": "x", "field": "fertility"},
          "y": {"scale": "y", "field": "life_expect"},
          "stroke": {"value": "#bbb"},
          "strokeWidth": {"value": 5},
          "strokeOpacity": {"value": 0.5}
        }
      }
    },
    {
      "name": "point",
      "type": "symbol",
      "from": {"data": "interpolate"},
      "encode": {
        "enter": {
          "fill": {"scale": "color", "field": "this.cluster"},
          "size": {"value": 150}
        },
        "update": {
          "x": {"scale": "x", "field": "inter_fertility"},
          "y": {"scale": "y", "field": "inter_life_expect"},
          "fillOpacity": [
            {
              "test": "datum.country==timeline.country || indata('trackCountries', 'country', datum.country)",
              "value": 1
            },
            {"value": 0.5}
          ]
        }
      }
    },
    {
      "type": "text",
      "from": {"data": "interpolate"},
      "interactive": false,
      "encode": {
        "enter": {
          "fill": {"value": "#333"},
          "fontSize": {"value": 14},
          "fontWeight": {"value": "bold"},
          "text": {"field": "country"},
          "align": {"value": "center"},
          "baseline": {"value": "bottom"}
        },
        "update": {
          "x": {"scale": "x", "field": "inter_fertility"},
          "y": {"scale": "y", "field": "inter_life_expect", "offset": -7},
          "fillOpacity": [
            {
              "test": "datum.country==timeline.country || indata('trackCountries', 'country', datum.country)",
              "value": 0.8
            },
            {"value": 0}
          ]
        }
      }
    }
  ]
});
var view = new vega.View(runtimeSpec)
  .initialize(document.querySelector('#view'))
  .renderer('svg')
  .hover()
  .run();
    </script>
  </body>
</html>

Overwriting f1.template


In [30]:
f1(w=1000,h=800)

In [31]:
import glob, os
for f in glob.glob("__tmp*.html"):
    print(f)
    os.remove(f)

__tmp12660_1.html
__tmp12660_2.html
__tmp12660_3.html
__tmp12660_4.html
